# Example 1: Specimen (Amplitudes)
This notebook runs on Google Colab.

In [ ]:
!apt-get install -qq xvfb
!git clone https://github.com/adtzlr/fiberreinforcedrubber.git -qq
!pip install ./fiberreinforcedrubber -qq

In [ ]:
import felupe as fem
import matplotlib.pyplot as plt
import numpy as np
import termtables as tt
from pypardiso import spsolve

import fiberreinforcedrubber as frr

frr.__version__

Geometry parameters

In [ ]:
H = 80  # mm
W = 60  # mm
thickness = 5  # mm
middle = 5  # mm
angle = 54.7546 * 2  # mm
radius = 42.3  # mm

Material properties of the two fiber families

In [ ]:
fiber_angle = 19  # deg
fiber_axis = 1  # axis from which the fiber-angle is measured
fiber_area = 0.08  # mm^2
fiber_modulus = 5500  # N/mm^2
fiber_distance = 1 / 0.95  # mm
strain_exponent = 1

Rubber material properties

In [ ]:
C10 = 0.5  # N/mm^2 (half shear modulus)

Deformation parameters

In [ ]:
lateral_max = 23
tension_max = 8

Plot options

In [ ]:
line_segments = 501  # number of lines per cord

Generate the meshes

In [ ]:
mesh, fibers_1, fibers_2, mask_points_1, mask_points_2 = frr.create_test_specimen(
    width=W,
    height=H,
    radius=radius,
    angle=angle,
    fiber_angle=fiber_angle,
    fiber_axis=fiber_axis,
    fiber_distance=fiber_distance,
    n=line_segments,
    middle=middle,
)

View the rubber mesh

In [ ]:
view = fem.ViewMesh(mesh)
plotter = view.plot(
    notebook=True,
    theme="document",
    add_axes=False,
    edge_color="black",
    color="lightgrey",
    line_width=4,
)
plotter.show(jupyter_backend="static")

View the fiber mesh

In [ ]:
view = fem.ViewMesh(fem.mesh.concatenate([fibers_1, fibers_2]))
plotter = view.plot(
    notebook=True,
    theme="document",
    add_axes=False,
    color="black",
    line_width=4,
)
plotter.show(jupyter_backend="static")

Create a numeric region and a displacement field

In [ ]:
region = fem.RegionQuad(mesh)
field = fem.FieldContainer([fem.Field(region, dim=2)])

Setup boundary conditions

In [ ]:
bounds, loadcase = fem.dof.shear(field)

Constitutive material behavior for rubber and cord

In [ ]:
neohooke, fibermat1, fibermat2, vector1, vector2 = frr.fiber_reinforced_rubber(
    C10=C10,
    fiber_angle=fiber_angle,
    fiber_modulus=fiber_modulus,
    fiber_area=fiber_area,
    thickness=thickness,
    strain_exponent=strain_exponent,
    axis=fiber_axis,
    fiber_distance=fiber_distance,
)

Solid bodies

In [ ]:
rubber = fem.SolidBody(neohooke, field)
fiber1 = fem.SolidBody(fibermat1, field)
fiber2 = fem.SolidBody(fibermat2, field)

Tension and shear at V = 3 mm (Fy = 3 kN)

In [ ]:
fiber_forces = [[], []]


def evaluate_fiber_forces(i, j, substep):
    field = substep.x

    # get fiber normal forces per undeformed (fiber) area
    force1 = frr.fiber_force(fiber1, thickness, fiber_area, vector1)
    force2 = frr.fiber_force(fiber2, thickness, fiber_area, vector2)

    # interpolate fiber forces to the line-meshes of the fiber families
    r_1 = frr.interpolate(mesh, force1, fibers_1, mask_points_1)
    r_2 = frr.interpolate(mesh, force2, fibers_2, mask_points_2)

    fiber_forces[0].append(r_1)
    fiber_forces[1].append(r_2)


step = fem.Step(
    items=[rubber, fiber1, fiber2],
    boundaries=bounds,
    ramp={
        bounds["compression_top"]: fem.math.linsteps([3, 3], num=2),
        bounds["move"]: lateral_max * fem.math.linsteps([-1, 1], num=2),
    },
)
job = fem.Job(steps=[step], callback=evaluate_fiber_forces)
job.evaluate(solver=spsolve, tol=1e-2)

Interpolate displacements to the line-meshes of the fiber families

In [ ]:
u_1 = frr.interpolate(mesh, field[0].values, fibers_1, mask_points_1)
u_2 = frr.interpolate(mesh, field[0].values, fibers_2, mask_points_2)

Deformed line mesh for the fibers

In [ ]:
fibers_1.points += u_1
fibers_2.points += u_2

## Postprocessing
Deformed Views

View on fiber families

In [ ]:
fiberfamilies = [
    (fiber_forces[0], fibers_1, fibers_2, [400, 900]),
    (fiber_forces[1], fibers_2, fibers_1, [400, 900]),
]
for i, (fiberforce, fiberfamily1, fiberfamily2, clim) in enumerate(fiberfamilies):
    view = fem.ViewSolid(field)
    plotter = view.plot(
        notebook=True,
        theme="document",
        show_edges=False,
        add_axes=False,
    )
    plotter.add_axes(label_size=(0.06, 0.06))

    forcerange = np.abs(fiberforce[-1] - fiberforce[-3])

    fiberview1 = fem.ViewMesh(
        fiberfamily1,
        point_data={
            "Normal Force (Range) per Undeformed Area (Fibre) in MPa": forcerange
        },
    )
    fiberview2 = fem.ViewMesh(
        fiberfamily2,
    )
    plotter = fiberview2.plot(
        plotter=plotter,
        line_width=3,
        add_axes=False,
    )
    fiberplotter = fiberview1.plot(
        "Normal Force (Range) per Undeformed Area (Fibre) in MPa",
        label="Normal Force (Range) per Undeformed Area (Fibre) in MPa",
        plotter=plotter,
        component=None,
        clim=clim,
        below_color="darkgrey",
        above_color="lightgrey",
        line_width=3,
        add_axes=False,
    )
    fiberplotter.show(jupyter_backend="static")